In [110]:
!pip install llama-index-embeddings-jinaai

In [111]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.embeddings.jinaai import JinaEmbedding
import os
import chromadb
from llama_index.core import Document, StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.core.settings import Settings
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import StorageContext
import logging
import sys




Note: you may need to restart the kernel to use updated packages.

In [112]:
from dotenv import load_dotenv
import os

# Deine .env / keys.env Datei laden
load_dotenv("/Users/perezopoku/Desktop/Programme/Persönlcihe Projekte/RAG/keys.env")

# API-Key korrekt auslesen
jinaai_api_key = os.getenv("JINAAI_API_KEY")

print(jinaai_api_key)


JINAAI_API_KEY


In [113]:
def load_Text(path):
    docs = []
    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            full_path = os.path.join(path, filename)
            with open(full_path, "r", encoding="utf-8") as f:
                docs.append((filename, f.read()))
    return docs

documents = load_Text("../Gesetze")
print(f"Gelesene Dateien: {len(documents)}")


Gelesene Dateien: 11


In [124]:
def chunk_by_tokens(text, max_tokens=8000):
    tokens = tokenizer.encode(text)
    chunks = []

    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i+max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)

    return chunks


print(chunks)



['Gesetz �ber die Zahlung des Arbeitsentgelts an Feiertagen und im Krankheitsfall\n\n(1) Dieses Gesetz regelt die Zahlung des Arbeitsentgelts an gesetzlichen Feiertagen und die Fortzahlung des Arbeitsentgelts im Krankheitsfall an Arbeitnehmer sowie die wirtschaftliche Sicherung im Bereich der Heimarbeit für gesetzliche Feiertage und im Krankheitsfall.\n(2) Arbeitnehmer in Sinne dieses Gesetzes sind Arbeiter und Angestellte sowie die zu ihrer Berufsbildung Beschäftigten.\n\n(1) Für Arbeitszeit, die infolge eines gesetzlichen Feiertages ausfällt, hat der Arbeitgeber dem Arbeitnehmer das Arbeitsentgelt zu zahlen, das er ohne den Arbeitsausfall erhalten hätte.\n(2) Die Arbeitszeit, die an einem gesetzlichen Feiertag gleichzeitig infolge von Kurzarbeit ausfällt und für die an anderen Tagen als an gesetzlichen Feiertagen Kurzarbeitergeld geleistet wird, gilt als infolge eines gesetzlichen Feiertages nach Absatz 1 ausgefallen.\n(3) Arbeitnehmer, die am letzten Arbeitstag vor oder am ersten Ar

In [116]:
Settings.embed_model = JinaEmbedding(
    api_key=os.getenv("JINAAI_API_KEY"),
    model="jina-embeddings-v2"   # Beispiel
)

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display
from llama_index.core import Document


INFO:backoff:Backing off send_request(...) for 0.5s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))
Backing off send_request(...) for 0.5s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))
Backing off send_request(...) for 0.5s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))


In [120]:
from llama_index.core import StorageContext


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    llama_docs,
    storage_context=storage_context,
    embed_model=text_embed_model)


In [ ]:
"""import os
import chromadb
from llama_index.core import Document, StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.core.settings import Settings
Settings.llm = None

#dokumente Laden

from llama_index.core import Document
tokenizer = tiktoken.get_encoding("cl100k_base")

def chunk_by_tokens(text, max_tokens=8000):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i+max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks

# ------------------ DOKUMENTE CHUNKEN ------------------

llama_docs = []

for filename, text in documents:
    chunks = chunk_by_tokens(text, max_tokens=8000)
    for chunk in chunks:
        doc = Document(
            text=chunk,
            metadata={"source": filename}
        )
        llama_docs.append(doc)

# ------------------ EMBEDDING-MODELL ------------------

text_embed_model = JinaEmbedding(
    api_key=os.environ["JINAAI_API_KEY"],
    model="jina-embeddings-v3",
    task="retrieval.passage",
)



#Jina Embedding Modell
text_embed_model = JinaEmbedding(
    api_key="JINAAI_API_KEY",
    model="jina-embeddings-v3",
    task="retrieval.passage",
)
# VektorDatenbank anlegen
db = chromadb.PersistentClient(
    path="../chroma_db"
)
chroma_collection = db.get_or_create_collection("quickstart")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_documents(
    llama_docs,
    storage_context=storage_context,
    embed_model=text_embed_model,
)


db2 = chromadb.PersistentClient(
    path="/RAG/chroma_db"
)
chroma_collection = db2.get_or_create_collection("quickstart")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=text_embed_model,
)


# Query

query_engine = index.as_query_engine(llm=None)  # wichtig: kein OpenAI LLM!
response = query_engine.query("Was bedeutet Diebstahl nach § 242 StGB?")

print(response)
"""

LLM is explicitly disabled. Using MockLLM.


RuntimeError: [RID: b99997f060ba726c78b286e31dcc1bbb] Invalid API key

In [ ]:
questions = ["Mein chef hat mich ohne Grund gefeuert, was kann ich tun?",
"wie viele Tage urlaub bekomme ich, mein chef sagt ich habe nur 2 wochen urlaub im Jahr",
" ich werde erst ab 17:15 bezahlt, aber mein Chef sagt ich muss um 17 uhr da sein, darf er so?"
]

In [ ]:
from pprint import pprint
for question in questions:
    response = query_engine.query(question)
    pprint(f"Frage: {question}\nAntwort: {response}\n")

ggml_metal_free: deallocating
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)


ReadTimeout: HTTPSConnectionPool(host='api.jina.ai', port=443): Read timed out. (read timeout=None)

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [126]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex
import chromadb
import os

# 1. Jina API Key eintragen
text_embed_model = JinaEmbedding(
    api_key="JINAAI_API_KEY",
    model="jina-embeddings-v3",
    task="retrieval.passage",
)
# 2. ChromaDB (neu erstellt, wenn leer)
db = chromadb.PersistentClient(
    path="/..chroma_db"
)
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 3. Index laden oder neu erstellen
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=text_embed_model,
)

# 4. Lokales LLM
llm = LlamaCPP(
    model_path="/Users/perezopoku/.lmstudio/models/RichardErkhov/meta-llama_-_Llama-3.2-1B-gguf/Llama-3.2-1B.Q8_0.gguf",
    temperature=0.2,
    max_new_tokens=256,
    context_window=4096,
)

# 5. Query Engine mit lokalem LLM
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query("Was bedeutet Diebstahl nach § 242 StGB?")
pprint(response)


NameError: name 'api_key' is not defined

In [ ]:
from pprint import pprint
for question in questions:
    response = query_engine.query(question)
    pprint(f"Frage: {question}\nAntwort: {response}\n")

Llama.generate: 10 prefix-match hit, remaining 903 prompt tokens to eval


KeyboardInterrupt: 